SVM

Logistic Regression

ANNs

KNN

Decision Tree

Naive Bayes





In [6]:
from datasets import popularity_dataset
popularity_data = popularity_dataset().dropna()
display(popularity_data)

,days_article_dataset,number_words_title,number_words_content,rate_unique_words_content,rate_non_stop_words_content,rate_unique_non_stop_words_content,number_links,number_links_to_mashable,number_imgs,number_videos,...,min_polarity_in_positive_words,max_polarity_in_positive_words,avg_polarity_in_negative_words,min_polarity_in_negative_words,max_polarity_in_negative_words,title_sentiment_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,731,12.0,219,0.663594,1.0,0.815385,4,2,1,0,...,0.100000,0.70,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,731,9.0,255,0.604743,1.0,0.791946,3,1,1,0,...,0.033333,0.70,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,731,9.0,211,0.575130,1.0,0.663866,3,1,1,0,...,0.100000,1.00,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,731,9.0,531,0.503788,1.0,0.665635,9,0,1,0,...,0.136364,0.80,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,731,13.0,1072,0.415646,1.0,0.540890,19,19,20,0,...,0.033333,1.00,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,8,11.0,346,0.529052,1.0,0.684783,9,7,1,1,...,0.100000,0.75,-0.260000,-0.500,-0.125000,0.100000,0.000000,0.400000,0.000000,1800
39640,8,12.0,328,0.696296,1.0,0.885057,9,7,3,48,...,0.136364,0.70,-0.211111,-0.400,-0.100000,0.300000,1.000000,0.200000,1.000000,1900
39641,8,10.0,442,0.516355,1.0,0.644128,24,1,12,1,...,0.136364,0.50,-0.356439,-0.800,-0.166667,0.454545,0.136364,0.045455,0.136364,1900
39642,8,6.0,682,0.539493,1.0,0.692661,10,1,1,0,...,0.062500,0.50,-0.205246,-0.500,-0.012500,0.000000,0.000000,0.500000,0.000000,1100


In [7]:
median = popularity_data.median()[' shares']
display(median)

threshold = median
popularity_data.loc[popularity_data[' shares'] > threshold, 'will be successfull'] = 1
popularity_data.loc[popularity_data[' shares'] <= threshold, 'will be successfull'] = 0
display(popularity_data)

1400.0

,days_article_dataset,number_words_title,number_words_content,rate_unique_words_content,rate_non_stop_words_content,rate_unique_non_stop_words_content,number_links,number_links_to_mashable,number_imgs,number_videos,...,max_polarity_in_positive_words,avg_polarity_in_negative_words,min_polarity_in_negative_words,max_polarity_in_negative_words,title_sentiment_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,will be successfull
0,731,12.0,219,0.663594,1.0,0.815385,4,2,1,0,...,0.70,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,0.0
1,731,9.0,255,0.604743,1.0,0.791946,3,1,1,0,...,0.70,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,0.0
2,731,9.0,211,0.575130,1.0,0.663866,3,1,1,0,...,1.00,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,1.0
3,731,9.0,531,0.503788,1.0,0.665635,9,0,1,0,...,0.80,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,0.0
4,731,13.0,1072,0.415646,1.0,0.540890,19,19,20,0,...,1.00,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39639,8,11.0,346,0.529052,1.0,0.684783,9,7,1,1,...,0.75,-0.260000,-0.500,-0.125000,0.100000,0.000000,0.400000,0.000000,1800,1.0
39640,8,12.0,328,0.696296,1.0,0.885057,9,7,3,48,...,0.70,-0.211111,-0.400,-0.100000,0.300000,1.000000,0.200000,1.000000,1900,1.0
39641,8,10.0,442,0.516355,1.0,0.644128,24,1,12,1,...,0.50,-0.356439,-0.800,-0.166667,0.454545,0.136364,0.045455,0.136364,1900,1.0
39642,8,6.0,682,0.539493,1.0,0.692661,10,1,1,0,...,0.50,-0.205246,-0.500,-0.012500,0.000000,0.000000,0.500000,0.000000,1100,0.0


In [12]:
subset = popularity_data.iloc[0:6000]
data = subset
x = data.drop(' shares', axis=1).drop('will be successfull', axis=1)
y = data['will be successfull']

In [16]:
from sklearn.model_selection import cross_val_score
def accuracy_score(classifier, x, y):
    scores = cross_val_score(classifier, x, y, cv=5)
    print("Accuracies: ",scores)
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

In [17]:
from sklearn.svm import SVC


svcClf = SVC()

accuracy_score(svcClf, x, y)

Accuracies:  [0.43166667 0.56833333 0.56666667 0.56833333 0.56833333]
Accuracy: 0.54 (+/- 0.11)
